Loading the Kedro extension for IPython

In [87]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


Loading libraries

In [88]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

Loading the DataFrame

In [89]:
games = catalog.load("games")

[10/01/24 20:47:55] INFO     Loading data from games (CSVDataset)...                            ]8;id=920174;file://c:\Users\diego\OneDrive\Documentos\Kedro\metagames_kedro\env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=737365;file://c:\Users\diego\OneDrive\Documentos\Kedro\metagames_kedro\env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

# Phase 3: Data Preparation

To begin working on preparing the data without altering the original set, we will create a copy of the DataFrame. This will allow us to make changes and transformations without risking losing the original information.

In [90]:
games_new = games.copy()

Now we are going to check the columns that we have inside our DataFrame and the data type of each one.

In [91]:
games_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67571 entries, 0 to 67570
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             67571 non-null  int64  
 1   game                   67571 non-null  object 
 2   link                   67571 non-null  object 
 3   release                67571 non-null  object 
 4   peak_players           67571 non-null  int64  
 5   positive_reviews       67571 non-null  int64  
 6   negative_reviews       67571 non-null  int64  
 7   total_reviews          67571 non-null  int64  
 8   rating                 67571 non-null  float64
 9   primary_genre          67561 non-null  object 
 10  store_genres           67514 non-null  object 
 11  publisher              67110 non-null  object 
 12  developer              67443 non-null  object 
 13  detected_technologies  60265 non-null  object 
 14  store_asset_mod_time   67275 non-null  object 
 15  re

We can see that the columns players_right_now and 24_hour_peak store data of the object type but their content is only numbers, so we will now change the data type of these columns to be able to have a better analysis.

In [92]:
# Replace commas with periods in columns
games_new['players_right_now'] = games_new['players_right_now'].str.replace(',', '.')
games_new['24_hour_peak'] = games_new['24_hour_peak'].str. replace(',', '.')

# Convert the columns to numeric type
games_new['players_right_now'] = pd.to_numeric(games_new['players_right_now'], errors='coerce')
games_new['24_hour_peak'] = pd. to_numeric(games_new['24_hour_peak'], errors='coerce')

## Deleting columns

In this phase, we will remove columns that are not relevant to our analysis, thus simplifying our DataFrame and improving the efficiency of data processing.

### Discard columns for very high correlations

In this subsection, we will proceed to remove columns from the dataset that have very high correlations with each other. Highly correlated variables can introduce redundancy into the model, which in turn can negatively affect the interpretation of the results and the efficiency of the model. Therefore, it is crucial to identify and remove these columns before proceeding with the analysis.

Next, we will present a code that calculates the correlation matrix and determines which columns have an absolute correlation greater than a specific threshold. For this report we chose a threshold of 0.9

In [93]:
def handling_correlation(df, threshold=0.9):

    # Filtering the DataFrame to work only with numeric columns
    numeric_df = df.select_dtypes(include='number')

    # Creating a set to store the correlated features
    corr_features = set()

    # Creating the correlation matrix (by default it uses Pearson)
    corr_matrix = numeric_df.corr()

    # Code to select all correlated features
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                corr_features.add(colname)

    return list(corr_features)

Now the handling_correlation function will be used to obtain the columns that exceed the defined threshold and these will be discarded from the DataFrame.

In [94]:
# Get the correlated columns
correlated_columns = handling_correlation(games_new)

# Remove the correlated columns from the original DataFrame
games_new = games_new.drop(columns=correlated_columns)

# Display the deleted columns
print("Columns : were deleted",correlated_columns)

Columns : were deleted ['all_time_peak', 'review_percentage', 'total_reviews']


### Discard columns by importance

We will now proceed with the elimination of columns based on their importance in the analysis. This not only improves interpretability but also optimizes the performance of the model by reducing dimensionality.

Before proceeding with the elimination, it is essential to highlight that the criteria used to discard these columns is based on several factors. First, we evaluate the relevance of each feature in relation to the final objective of the analysis, ensuring that the retained columns add value to the prediction of user affinity with the games. In addition, we consider the quality and completeness of the data, eliminating those features with high levels of null or inconsistent values. Finally, we prioritize reducing redundancy by keeping only the most representative metrics to avoid duplication of information.

- link: The link to the store does not provide any relevant information for predicting the affinity of a game. It is purely identifying data that does not contribute to improving the analysis of user behavior or the game.

- store_genres: While the genres category may seem useful, there is already a main genres column that probably captures all the relevant information. This "additional genres" column introduces redundant information and could create noise by duplicating or complicating category analysis.

- detected_technologies: sdsd This column contains technical information about the technologies used by the game, such as the graphics engine or the technologies behind the development. However, this data does not directly influence the affinity or user experience when selecting games, as players generally do not choose games based on the graphics engine used. Additionally, a large number of null values ​​suggests that the information is not consistent or relevant for all games.

- store_asset_mod_time: The date when the game content was updated in the store is not a factor that affects the user's affinity with the game. Furthermore, it is a data point that is more related to store management than to player experience. Having a significant amount of null values ​​makes it even less useful for analysis, as there is no complete coverage of all games.

- 24_hour_peak: The peak_players column is now being retained, containing the all-time high number of concurrent players, a more robust metric that represents the game’s popularity over time. The presence of null values ​​in this column indicates that there is no consistent data across all games, making its removal easier to analyze without significant loss of useful information.

- all_time_peak_date: The date when the highest number of concurrent players was reached is a historical data point that, while interesting from the point of view of the game's popularity in the past, has no direct relation to a user's current affinity. Furthermore, this type of data does not contribute to the predictive model, since user interest depends more on intrinsic characteristics of the game than on historical events.

In [95]:
# Listado de columnas a eliminar
columns_to_drop = [
    'link', 
    'store_genres', 
    'detected_technologies', 
    'store_asset_mod_time', 
    '24_hour_peak', 
    'all_time_peak_date'
]

# Eliminar las columnas del DataFrame
games_new = games_new.drop(columns=columns_to_drop)

To ensure that we have successfully removed the unwanted columns, we proceed to check the remaining columns in the new DataFrame again.

In [96]:
# Print column names one below the other
for column in games_new.columns:
    print(column)

Unnamed: 0
game
release
peak_players
positive_reviews
negative_reviews
rating
primary_genre
publisher
developer
players_right_now


## Handling null data

We check if there are null values ​​and which are the columns with null values

In [97]:
games_new.isnull().any()


Unnamed: 0           False
game                 False
release              False
peak_players         False
positive_reviews     False
negative_reviews     False
rating               False
primary_genre         True
publisher             True
developer             True
players_right_now     True
dtype: bool

We show how many null values ​​there are per column

In [98]:
# Calculate the number of null values
null_values = games_new.isnull().sum()

# Filter the values
null_values = null_values[null_values > 0]

print (null_values)

primary_genre         10
publisher            461
developer            128
players_right_now      6
dtype: int64


We check where null values ​​exist using a heat map, making it easy to identify rows and columns with missing data.

### Data imputation column 'Primary_genre'

#### Check the number of null values ​​in the 'Primary Genre' column

The number of missing values ​​in the 'primary_genre' column is counted.

This step is critical to understanding the magnitude of the missing data problem and being able to properly plan how to address it.

In [99]:
null_count_primary_genre = games_new['primary_genre'].isnull().sum()
print(f"Amount of null data in 'primary_genre': {null_count_primary_genre}")

Amount of null data in 'primary_genre': 10


#### Check for rows with null values ​​in 'Primary Genre' column

Here we identify the rows where the value for 'primary genre' is missing.

This is important because it helps us decide how to fill those values, taking into account the context of each game.

In [100]:
# Filter rows with null values ​​in 'primary_genre'
nulos_primary_genre = games_new[games_new['primary_genre'].isnull()]

# Display game name for rows with null values ​​in 'primary_genre'
print(nulos_primary_genre[['game']])

                                        game
19978                              RunnerDot
35300                         Hentai Warfare
36097               My Secret Girlfriends ;)
36265                          MiniFarm 2020
46069        Redemption: Tyranny of Daetorem
49296                 SpaBerry VR Experience
49298                               Subsiege
60000                         The Spookening
60359                                Unfazed
65778  Defense Grid 2: A Matter of Endurance


#### Manually Assigning Genres to Games with Null Values

We manually assign a genre to each game that has a null value in the 'primary_genre' column.

For this, we use Pandas' .loc[] method, which allows us to assign a specific value to nulls depending on the row.

We make this decision based on our knowledge of the game's content or the most logical categorization. This ensures that each game has an assigned genre, which is important for future analysis.

In [101]:
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 19978 ), 'primary_genre'] = 'Shooter'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 35300), 'primary_genre'] = 'Adult Content'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 36097), 'primary_genre'] = 'Adult Content'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 36265 ), 'primary_genre'] = 'Simulation'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 46069 ), 'primary_genre'] = 'RPG'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 49296), 'primary_genre'] = 'Simulation'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 49298 ), 'primary_genre'] = 'Strategy'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 60000), 'primary_genre'] = 'Adventure'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 60359 ), 'primary_genre'] = 'Survival Horror'
games_new.loc[games_new['primary_genre'].isnull() & (games_new.index == 65778 ), 'primary_genre'] = 'Tower Defense'

#### Check if there is still null data in the primary genre column

Once we have verified that the number of null values ​​in the 'Primary_genre' column is non-existent, we will continue with the following imputations.

In [102]:
null_count_primary_genre = games_new['primary_genre'].isnull().sum()
print(f"Amount of null data in 'primary_genre'': {null_count_primary_genre}")

Amount of null data in 'primary_genre'': 0


### Data imputation column 'developer'

#### Check for rows with null values ​​in column 'developer'

We count the number of null values ​​in the 'developer' column

In [103]:
null_count = games_new['developer'].isnull().sum()
print(f"Amount of null data in 'developer'': {null_count}")

Amount of null data in 'developer'': 128


#### Check the Contents of Rows with Null Values

We filter out rows where the value of 'developer' is null to better understand the context of this missing data.

This allows us to decide how to impute the values ​​appropriately.

In [104]:
# Filter and show only rows where 'developer' is null
null_developer_rows = games_new.loc[games_new['developer'].isnull()]

# Show the result
null_developer_rows.head(100)

,Unnamed: 0,game,release,peak_players,positive_reviews,negative_reviews,rating,primary_genre,publisher,developer,players_right_now
243,243,Flashback,2023-02-20,119,153,13,83.13,Action (1),Ubisoft,NaN,0.0
246,246,Flashback,2019-02-28,41,326,77,75.82,Action (1),Ubisoft,NaN,0.0
249,249,Flashback,2013-10-01,236,275,200,56.66,Action (1),Ubisoft,NaN,0.0
4919,4919,Lords of Ravage - Сhapter 1,2022-09-08,22,34,1,81.11,RPG (3),NaN,NaN,0.0
5494,5494,Drill Deal - Oil Tycoon,2022-03-09,128,97,13,78.93,Strategy (2),Manager Games S.A.,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
61342,61342,Deathwave,2016-06-09,153,13,19,43.90,Action (1),NaN,NaN,0.0
61525,61525,The Gleam: VR Escape the Room,2016-10-14,896,18,36,38.32,Indie (23),NaN,NaN,0.0
61665,61665,Streamer Simulator,2016-08-23,14,10,39,29.52,Casual (4),NaN,NaN,0.0
62472,62472,Forge Quest,2015-05-29,493,430,109,75.30,Indie (23),Open Realms LLC,NaN,0.0


#### Imputation Based on 'publisher' Column

Since in most cases the publisher of the game is also its developer, we use the data in the 'publisher' column to fill in the null values ​​in the 'developer' column.

This allows us to leverage existing information in a logical and efficient way to fill in missing data.

In [105]:
# Copy the contents of 'publisher' to 'developer' only where 'developer' is null
games_new.loc[games_new['developer'].isnull(), 'developer'] = games_new['publisher']

# Check the result
print(games_new[['developer']].isnull().sum())

developer    86
dtype: int64


#### Filling Remaining Null Values ​​in Column 'developer'

After using the available information to fill in the null values, there is still some missing data in the 'developer' column.

To ensure that there are no null values, we populate those fields with a default value, in this case, "unknown".

This is useful to maintain the integrity of the dataset and avoid problems in subsequent analysis.

In [106]:
# Fill null values ​​in 'developer' column with "unknown"
games_new['developer'].fillna('unknown', inplace=True)

                    WARNING  C:\Users\diego\AppData\Local\Temp\ipykernel_22148\425896862.py:2:      ]8;id=107687;file://C:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\warnings.py\warnings.py]8;;\:]8;id=464734;file://C:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\warnings.py#112\112]8;;\
                             FutureWarning: A value is trying to be set on a copy of a DataFrame or                
                             Series through chained assignment using an inplace method.                            
                             The behavior will change in pandas 3.0. This inplace method will never                
                             work because the intermediate object on which we are setting values                   
                             always behaves as a copy.                                                             
                                                                                                                   
                             For example, when doing 'df[col].method(value, inplace=True)', try                    
                             using 'df.method({col: value}, inplace=True)' or df[col] =                            
                             df[col].method(value) instead, to perform the operation inplace on the                
                             original object.                                                                      
                                                                                                                   
                                                                                                                   
                               games_new['developer'].fillna('unknown', inplace=True)                              
                                                                                                                   

#### Verification of Imputation

We double-check the number of null values ​​in the 'developer' column to ensure that all missing data has been correctly imputed.

The expected result is zero, which would indicate that there are no more null values.

In [107]:
null_count = games_new['developer'].isnull().sum()
print(f"Amount of null data in 'developer'': {null_count}")

Amount of null data in 'developer'': 0


### Deleting the publisher column

After using the information from the 'publisher' column to fill in the missing data in 'developer', the 'publisher' column is no longer needed in our DataFrame. So, we will proceed to remove it.

In [108]:
games_new.drop(columns=['publisher'], inplace=True)

With this elimination, our dataset becomes cleaner and easier to handle by removing redundant information.

### Data Imputation Column 'players_right_now'

We count the number of null values ​​in the 'players_right_now' column

In [109]:
null_count_players_right_now = games_new['players_right_now'].isnull().sum()
print(f"Amount of null data in 'players_right_now': {null_count_players_right_now}")

Amount of null data in 'players_right_now': 6


#### K-Nearest Neighbors (KNN) Model

KNN is a supervised learning algorithm used for both classification and regression. In the context of data imputation, KNN estimates missing values ​​based on similarity to other data in the dataset. It works by identifying the 'k' closest data points (neighbors) to a point with a missing value and uses the values ​​from those neighbors to estimate the missing value.

#### Why do we use KNN for imputation?

- Improved Accuracy: KNN takes into account multiple features to find similarities, which can result in more accurate imputation compared to simpler methods.

- Flexibility: It can handle both numerical and categorical data, making it suitable for mixed data sets.

- Consistency: By using similar data, consistency is maintained across the data set, which is crucial for further analysis.

#### KNN Imputation Process for 'players_right_now'

1. Create a Temporary Copy of the DataFrame

In [110]:
# We create a copy of the DataFrame to avoid modifying the original data during the imputation process.
games_new_temp = games_new.copy()

2. Identify Numeric and Categorical Columns

In [111]:
# We separate the numerical and categorical columns to treat them appropriately during imputation.
numeric_cols = games_new.select_dtypes(include=['number']).columns
categorical_cols = games_new.select_dtypes(include=['object']).columns

3. Encoding Categorical Columns

In [112]:
# We convert the categorical columns into numerical values ​​so that they can be processed by the KNN algorithm.
label_encoder = LabelEncoder()
for col in categorical_cols:
    games_new_temp[col] = label_encoder.fit_transform(games_new_temp[col])

4. Prepare Data for Imputation

In [113]:
# We combine the coded numerical and categorical columns to create a dataset ready for imputation.
data_for_imputation = games_new_temp[numeric_cols.tolist() + categorical_cols.tolist()]

We use the KNN Imputer to estimate the missing values.

In [114]:
# Here, 'n_neighbors=5' indicates that we are considering the 5 nearest neighbors for imputation.
imputer = KNNImputer(n_neighbors=5)
data_imputed = imputer.fit_transform(data_for_imputation)

Replace Values ​​in the Original Column

In [115]:
# We replace the missing values ​​in the original column with the imputed values.
games_new['players_right_now'] = data_imputed[:, games_new_temp.columns.get_loc('players_right_now')]

Upon completion of this process, we expect all null values ​​in the 'players_right_now' column to be replaced with accurate estimates, thus improving the quality and completeness of the dataset for future analysis.

#### Verification of imputation with KNN

We count the number of null values ​​in the 'players_right_now' column that should be non-existent.

In [116]:
null_count_players_right_now = games_new['players_right_now'].isnull().sum()
print(f"Amount of null data in 'players_right_now': {null_count_players_right_now}")

Amount of null data in 'players_right_now': 0


## Comparison with original data

In this section, a comparison is presented between the original (games) and cleaned (games_new) DataFrames to evaluate the transformations performed and how they affect the data. Null values ​​will be analyzed and descriptive statistics will be summarized to highlight the differences and improvements in the new dataset.

### Null Value Comparison

First, null values ​​were analyzed in both DataFrames:

In [117]:
games.isnull().any()


Unnamed: 0               False
game                     False
link                     False
release                  False
peak_players             False
positive_reviews         False
negative_reviews         False
total_reviews            False
rating                   False
primary_genre             True
store_genres              True
publisher                 True
developer                 True
detected_technologies     True
store_asset_mod_time      True
review_percentage         True
players_right_now         True
24_hour_peak              True
all_time_peak            False
all_time_peak_date        True
dtype: bool

In [118]:
games_new.isnull().any()


Unnamed: 0           False
game                 False
release              False
peak_players         False
positive_reviews     False
negative_reviews     False
rating               False
primary_genre        False
developer            False
players_right_now    False
dtype: bool

Observations:

- In the original DataFrame (games), columns with null values ​​were found that may affect the analysis, indicating the need for data cleaning.

- In the cleaned DataFrame (games_new), unnecessary columns have been removed and missing values ​​have been imputed, resulting in a more complete and reliable dataset.

### Comparison of Descriptive Statistics

The descriptive statistics of both DataFrames were then compared:

In [119]:
games.describe()

,Unnamed: 0,peak_players,positive_reviews,negative_reviews,total_reviews,rating,review_percentage,all_time_peak
count,67571.000000,6.757100e+04,6.757100e+04,67571.000000,6.757100e+04,67571.000000,47767.000000,6.757100e+04
mean,33785.000000,9.528674e+02,1.273527e+03,216.893801,1.490421e+03,65.288208,74.840497,9.629879e+02
std,19506.211857,1.979093e+04,2.955163e+04,5434.959528,3.400923e+04,14.059623,18.764257,2.017869e+04
min,0.000000,0.000000e+00,0.000000e+00,0.000000,1.000000e+00,15.090000,0.000000,0.000000e+00
25%,16892.500000,3.000000e+00,5.000000e+00,1.000000,7.000000e+00,56.530000,64.000000,3.000000e+00
50%,33785.000000,7.000000e+00,1.900000e+01,6.000000,2.600000e+01,66.760000,79.000000,7.000000e+00
75%,50677.500000,4.600000e+01,9.900000e+01,29.000000,1.320000e+02,75.640000,90.000000,4.700000e+01
max,67570.000000,3.257248e+06,6.307931e+06,927317.000000,7.117776e+06,97.540000,100.000000,3.257248e+06


In [120]:
games_new.describe()

,Unnamed: 0,peak_players,positive_reviews,negative_reviews,rating,players_right_now
count,67571.000000,6.757100e+04,6.757100e+04,67571.000000,67571.000000,67571.000000
mean,33785.000000,9.528674e+02,1.273527e+03,216.893801,65.288208,19578.111986
std,19506.211857,1.979093e+04,2.955163e+04,5434.959528,14.059623,11331.274323
min,0.000000,0.000000e+00,0.000000e+00,0.000000,15.090000,0.000000
25%,16892.500000,3.000000e+00,5.000000e+00,1.000000,56.530000,9711.000000
50%,33785.000000,7.000000e+00,1.900000e+01,6.000000,66.760000,19449.000000
75%,50677.500000,4.600000e+01,9.900000e+01,29.000000,75.640000,29453.000000
max,67570.000000,3.257248e+06,6.307931e+06,927317.000000,97.540000,39409.000000


When comparing the descriptive statistics of the games and games_new DataFrames, it was observed that despite the removal of several columns in the cleaned DataFrame (games_new), the statistics (such as mean, standard deviation, minimum and maximum values) of the remaining columns remain almost identical to those of the original DataFrame (games).

This suggests that:

- Data Quality: Data cleansing was performed effectively, removing only the columns that did not provide analytical value. The columns that remained in the new DataFrame are robust and contain relevant information, ensuring that data integrity has been maintained.

- Impact of Cleanup: Removing unnecessary columns has simplified the dataset without affecting the statistical representation of important variables. This allows future analysis and modeling to focus on the features that truly influence game performance and relevance.

- Consistency in Information: The persistence of statistics between the two DataFrames indicates that the underlying data has remained constant, which reinforces the reliability of the results obtained from this new data set.